In [ ]:
#ライブラリのインストール
!pip install numpy
!pip install pandas
!pip install scipy
!pip install scikit-learn
!pip install gensim

In [ ]:
#ライブラリのインポート
import numpy as np
import pandas as pd

In [ ]:
# train.csvを読み込んで、データフレームに格納するためのプログラム
train_df = pd.read_csv('C:/Users/isk/workspace/2022/fake-news/train.csv')

In [ ]:
#モデルの構築・予測
#データの準備
from sklearn.model_selection import train_test_split

# 学習用のデータとテスト用のデータを分けるコード
# train_df['text']は文章、train_df['label']はそのラベルとなっているので、これらを使用します。
X_train, X_test, y_train, y_test = train_test_split(
    train_df['text'].values.astype('str'), train_df['label'].values.astype('int'), test_size=0.2, random_state=0)

In [ ]:
#特徴量の選出
#文章からTF-IDFへの変換
from sklearn.feature_extraction.text import TfidfVectorizer

# 70%以上出てきている単語を消去し、英語のストップワード辞書に含まれている単語も削除
vectrizer = TfidfVectorizer(stop_words='english', max_df=0.7)

#学習用・テスト用それぞれの文章をTF-IDFの値に変換
tfidf_train = vectrizer.fit_transform(X_train).toarray()
tfidf_test = vectrizer.transform(X_test).toarray()

In [ ]:
#文章からDoc2Vecへの変換
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# 学習用・テスト用それぞれにおいて単語を要素に持つリストを作成
word_train = [text.split(' ') for text in X_train]
word_test = [text.split(' ') for text in X_test]

# 学習用の文章を学習させる
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(word_train)]
model = Doc2Vec(documents, vector_size=300, window=20, min_count=5, workers=30)

# 学習させた結果から、学習用・テスト用の文章をベクトル表現に変換
doc_train = [model.infer_vector(word) for word in word_train]
doc_test = [model.infer_vector(word) for word in word_test]

In [ ]:
#学習
from sklearn.linear_model import LogisticRegression

logist_tfidf = LogisticRegression()
logist_doc = LogisticRegression()

# ロジスティック回帰モデルによってTF-IDFを学習させる。
logist_tfidf.fit(tfidf_train, y_train)

# ロジスティック回帰モデルによってDoc2Vecを学習させる
logist_doc.fit(doc_train, y_train)

In [ ]:
#予測
# 学習したモデルを用いてTF-IDFから予測する。
y_pred_tfidf = logist_tfidf.predict(tfidf_test)

# 学習したモデルを用いてDoc2Vecから予測する。
y_pred_doc = logist_doc.predict(doc_test)

In [ ]:
#結果の確認
from sklearn.metrics import confusion_matrix

def make_conf_matrix(y_true: list, y_pred: list):
    '''
    実際のラベルと予測したラベルから、混同行列をデータフレームとして表示する関数。
    '''
    cm = confusion_matrix(y_true, y_pred)

    cm_dict = {
      'フェイクニュース': {
        'フェイクニュース判定': cm[1][1],
        '正常なニュース判定': cm[1][0]
      },
      '正常なニュース':{
        'フェイクニュース判定': cm[0][1],
        '正常なニュース判定': cm[0][0]
      }
    }

    df = pd.DataFrame(cm_dict)
    return df

In [ ]:
# TF-IDFを用いた結果を表示
make_conf_matrix(y_test, y_pred_tfidf)

,フェイクニュース,正常なニュース
フェイクニュース判定,2010,125
正常なニュース判定,104,1921


In [ ]:
# Doc2Vecを用いた結果を表示
make_conf_matrix(y_test, y_pred_doc)

,フェイクニュース,正常なニュース
フェイクニュース判定,2060,148
正常なニュース判定,54,1898


In [ ]:
#正解率,適合率,再現率,F1値の確認　
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

def show_scores(y_true: list, y_pred: list):
    '''
    実際のラベルと予測したラベルから、正解率・適合率・再現率・F1値を出力する関数。
    ''' 
    #それぞれのスコアを出すためのコード 
    acc_score = accuracy_score(y_true, y_pred)
    prec_score = precision_score(y_true, y_pred)
    rec_score = recall_score(y_true, y_pred)
    f_score = f1_score(y_true, y_pred)
    
    print('正解率: {:.2f}%'.format(acc_score*100))
    print('適合率: {:.2f}%'.format(prec_score*100))
    print('再現率: {:.2f}%'.format(rec_score*100))
    print('F1値: {:.2f}%'.format(f_score*100))

In [ ]:
#TF-IDFを用いた結果の可視化
show_scores(y_test, y_pred_tfidf)

正解率: 94.50%
適合率: 94.15%
再現率: 95.08%
F1値: 94.61%


In [ ]:
#Doc2Vecを用いた結果の可視化
show_scores(y_test, y_pred_doc)

正解率: 95.14%
適合率: 93.30%
再現率: 97.45%
F1値: 95.33%
